In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# импортируем класс TransformerMixin, позволяющий
# написать собственные классы
from sklearn.base import TransformerMixin
# импортируем класс ColumnTransformer, позволяющий выполнять
# преобразования для отдельных типов столбцов
from sklearn.compose import ColumnTransformer
# импортируем класс Pipeline, позволяющий создавать конвейеры
from sklearn.pipeline import Pipeline
# импортируем класс GridSearchCV, позволяющий 
# выполнить решетчатый поиск
from sklearn.model_selection import GridSearchCV
# импортируем класс SimpleImputer, позволяющий
# выполнить импутацию пропусков
from sklearn.impute import SimpleImputer
# импортируем класс PowerTransformer, позволяющий выполнить 
# преобразование Бокса-Кокса/Йео-Джонсона и стандартизацию
from sklearn.preprocessing import PowerTransformer
# импортируем класс OneHotEncoder, позволяющий
# выполнить дамми-кодирование
from sklearn.preprocessing import OneHotEncoder
# импортируем класс KFold и функцию cross_val_score
# для выполнения перекрестной проверки
from sklearn.model_selection import KFold, cross_val_score
# импортируем класс LogisticRegression для построения
# логистической регрессии
from sklearn.linear_model import LogisticRegression

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/Verizon_miss_dubl.csv', sep=';')

In [3]:
# смотрим первые 5 наблюдений
data.head()

,longdist,internat,local,int_disc,billtype,pay,age,gender,marital,children,income,churn
0,"27,09",0,"39,74",Нет,Бюджетный,CC,35.0,Женский,Женат,0.0,77680,0
1,NaN,0,"46,31",Нет,NaN,NaN,53.0,Мужской,Одинокий,1.0,"37111,5",0
2,"23,76",0,NaN,NaN,Бюджетный,Auto,NaN,Женский,NaN,1.0,"37079,4",0
3,"9,4",NaN,"13,9",Нет,NaN,CH,NaN,Мужской,Одинокий,NaN,81997,0
4,"14,15",0,"108,43",Да,Бесплатный,Auto,39.0,Женский,Одинокий,0.0,"16829,6",0


In [4]:
# смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4431 entries, 0 to 4430
Data columns (total 12 columns):
longdist    4430 non-null object
internat    4427 non-null object
local       4428 non-null object
int_disc    4430 non-null object
billtype    4427 non-null object
pay         4429 non-null object
age         4428 non-null float64
gender      4430 non-null object
marital     4427 non-null object
children    4430 non-null float64
income      4430 non-null object
churn       4431 non-null int64
dtypes: float64(2), int64(1), object(9)
memory usage: 415.5+ KB


In [5]:
# заменяем запятые на точки и преобразуем в тип float
for i in ['longdist', 'internat', 'local', 'income']:
    data[i] = data[i].str.replace(',', '.').astype('float')

In [6]:
# создаем список категориальных переменных
cat_cols = data.select_dtypes(include=['object']).columns.tolist()
# смотрим уникальные значения этих переменных
for i in cat_cols:
    print(i, data[i].unique())

int_disc ['Нет' nan 'Да']
billtype ['Бюджетный' nan 'Бесплатный']
pay ['CC' nan 'Auto' 'CH' 'CD']
gender ['Женский' 'Мужской' nan 'Женский&*' 'Мужской&*']
marital ['Женат' 'Одинокий' nan '_Одинокий' '_Женат' 'Же&нат']


In [7]:
# удаляем лишние символы в категориях переменных
# gender и marital
for i in ['gender', 'marital']:
    data[i] = data[i].str.replace('[*&_]', '')
    
# проверяем
for i in ['gender', 'marital']:
    print(i, data[i].unique())

gender ['Женский' 'Мужской' nan]
marital ['Женат' 'Одинокий' nan]


In [8]:
# смотрим частоты по категориальным переменным, 
# чтобы выявить редкие категории
for i in cat_cols:
    print(i)
    print('')
    print(data[i].value_counts(dropna=False))

int_disc

Нет    3054
Да     1376
NaN       1
Name: int_disc, dtype: int64
billtype

Бюджетный     2244
Бесплатный    2183
NaN              4
Name: billtype, dtype: int64
pay

CC      2561
CH       977
Auto     889
CD         2
NaN        2
Name: pay, dtype: int64
gender

Женский    2244
Мужской    2186
NaN           1
Name: gender, dtype: int64
marital

Женат       2625
Одинокий    1802
NaN            4
Name: marital, dtype: int64


In [9]:
# удаляем дубли на месте, оставляя первое
# встретившееся наблюдение в паттерне дубля
data.drop_duplicates(subset=None, keep='first', 
                     inplace=True)

In [10]:
# заменяем редкую категорию модой
data.at[data['pay'] == 'CD', 'pay'] = 'CC'

In [11]:
# создаем переменную - результат конъюнкции
data['gender_marital'] = data.apply(
    lambda x: f"{x['gender']} + {x['marital']}", 
    axis=1)

In [12]:
# поделим возраст на длительность междугородних звонков в минутах
data['ratio'] = data['age'] / data['longdist']
# заменяем бесконечные значения на 1
data['ratio'].replace([np.inf, -np.inf], 1, inplace=True)

In [13]:
# поделим длительность междугородних звонков в минутах на
# длительность международных звонков в минутах
data['ratio2'] = data['longdist'] / data['internat']
# заменяем бесконечные значения на 0
data['ratio2'].replace([np.inf, -np.inf], 0, inplace=True)

In [14]:
# поделим доход на возраст
data['ratio3'] = data['income'] / data['age']
# заменяем бесконечные значения на 0
data['ratio3'].replace([np.inf, -np.inf], 0, inplace=True)

In [15]:
# поделим возраст на количество детей
data['ratio4'] = data['age'] / data['children']
# заменяем бесконечные значения на 0
data['ratio4'].replace([np.inf, -np.inf], 0, inplace=True)

In [16]:
# разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data.drop('churn', axis=1), 
                                                    data['churn'],
                                                    test_size=0.3,
                                                    stratify=data['churn'],
                                                    random_state=42)

In [17]:
# создаем список категориальных переменных, список количественных 
# переменных, не предназначенных для биннинга, список 
# количественных переменных, предназначенных для биннинга
cat_columns = X_train.dtypes[X_train.dtypes == 'object'].index
num_columns = X_train.dtypes[X_train.dtypes != 'object'].index
num_columns_for_bin = ['age']

In [18]:
# создаем собственный класс,
# выполняющий биннинг
class CustomBinning(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        bins = np.array([-np.inf,  30, 50, np.inf])
        X_bin = np.digitize(X, bins)
        return X_bin

In [19]:
# создаем собственный класс,
# заменяющий 0 на 0.5
class Replacer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_replaced = np.where(X == 0, 0.5, X) 
        return X_replaced

In [20]:
# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [21]:
# создаем конвейер для количественных переменных, 
# не предназначенных для биннинга
num_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('replacer', Replacer()),
    ('boxcox', PowerTransformer(method='box-cox', standardize=True))
])

In [22]:
# создаем конвейер для количественных переменных, 
# предназначенных для биннинга
num_bin_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('bin', CustomBinning()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [23]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('num_bin', num_bin_pipe, num_columns_for_bin),
                ('cat', cat_pipe, cat_columns)]

# передаем список в ColumnTransformer
ct = ColumnTransformer(transformers=transformers)

# задаем итоговый конвейер
ml_pipe = Pipeline([('transform', ct), 
                    ('logreg', LogisticRegression(solver='liblinear'))])

In [24]:
# задаем сетку гиперпараметров
param_grid = {
    'transform__num__imputer__strategy': ['mean', 'median', 'constant'],    
    'transform__cat__imputer__strategy': ['most_frequent', 'constant'],
    'logreg__C': [.01, .1, .5, 1, 5, 10, 100]
}
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs = GridSearchCV(ml_pipe, 
                  param_grid,
                  cv=5, 
                  return_train_score=False)
# выполняем решетчатый поиск
gs.fit(X_train, y_train)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(
    gs.best_params_))
# смотрим наилучшее значение правильности
print('Наилучшее значение правильности: {:.3f}'.format(
    gs.best_score_))
# смотрим правильность на тестовой выборке
print('Значение правильности на тестовой выборке: {:.3f}'.format(
    gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'logreg__C': 0.5, 'transform__cat__imputer__strategy': 'most_frequent', 'transform__num__imputer__strategy': 'median'}
Наилучшее значение правильности: 0.812
Значение правильности на тестовой выборке: 0.805


In [25]:
# записываем CSV-файл в объект DataFrame 
fulldata = pd.read_csv('Data/Verizon_miss_dubl.csv', sep=';')

In [26]:
# пишем функцию, выполняющую предварительную обработку исторических данных
def preprocessing(df):
    # заменяем запятые на точки и преобразуем в тип float
    for i in ['longdist', 'internat', 'local', 'income']:
        df[i] = df[i].str.replace(',', '.').astype('float')
    # удаляем возможные лишние символы (все символы, не являющиеся 
    # буквами, символы нижнего подчеркивания и цифры) в категориях 
    # переменных gender и marital
    for i in ['gender', 'marital']:
        df[i] = df[i].str.replace('[\d+\W_]', '')
    # удаляем дубли на месте, оставляя первое
    # встретившееся наблюдение в паттерне дубля
    df.drop_duplicates(subset=None, keep='first', inplace=True)
    # все новые категории переменной pay заменяем модой
    lst = ['CC', 'Auto', 'CH', np.NaN]
    replace_new_values = lambda x: 'CC' if x not in lst else x
    df['pay'] = df['pay'].map(replace_new_values)
    # создаем переменную - результат конъюнкции
    df['gender_marital'] = df.apply(
        lambda x: f"{x['gender']} + {x['marital']}", 
        axis=1)                    
    # поделим возраст на длительность междугородних звонков в минутах
    df['ratio'] = df['age'] / df['longdist']
    # заменяем бесконечные значения на 1
    df['ratio'].replace([np.inf, -np.inf], 1, inplace=True)
    # поделим длительность междугородних звонков в минутах на
    # длительность международных звонков в минутах
    df['ratio2'] = df['longdist'] / df['internat']
    # заменяем бесконечные значения на 0
    df['ratio2'].replace([np.inf, -np.inf], 0, inplace=True)
    # поделим доход на возраст
    df['ratio3'] = df['income'] / df['age']
    # заменяем бесконечные значения на 0
    df['ratio3'].replace([np.inf, -np.inf], 0, inplace=True)
    # поделим возраст на количество детей
    df['ratio4'] = df['age'] / df['children']
    # заменяем бесконечные значения на 0
    df['ratio4'].replace([np.inf, -np.inf], 0, inplace=True)

In [27]:
# применяем функцию предварительной обработки 
# ко всем историческим данным
preprocessing(fulldata)

In [28]:
# создаем массив меток и массив признаков
y_fulldata = fulldata.pop('churn').values

In [29]:
# модифицируем конвейеры с учетом найденных 
# оптимальных значений гиперпараметров
cat_pipe_upd = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

num_pipe_upd = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('replacer', Replacer()),
    ('boxcox', PowerTransformer(method='box-cox', standardize=True))
])

In [30]:
# создаем список трансформеров
transformers_full = [('num', num_pipe_upd, num_columns),
                     ('num_bin', num_bin_pipe, num_columns_for_bin),
                     ('cat', cat_pipe_upd, cat_columns)]

# передаем список в ColumnTransformer
ct_full = ColumnTransformer(transformers=transformers_full)

# обучаем трансформеры с оптимальными значениями гиперпараметров
# на всех исторических данных и выполняем преобразования
fulldata = ct_full.fit_transform(fulldata)

In [31]:
# проверяем размерность
fulldata.shape

(1487, 31)

In [32]:
# обучаем модель логистической регрессии с оптимальными значениями
# гиперпараметров (C=0.5) на всех исторических данных
logreg_full = LogisticRegression(solver='liblinear', C=0.5, max_iter=200).fit(fulldata, y_fulldata)
# печатаем значение правильности
print('Правильность на всей исторической выборке: {:.3f}'.format(
    logreg_full.score(fulldata, y_fulldata)))

Правильность на всей исторической выборке: 0.807


In [33]:
# записываем CSV-файл, содержащий новые данные,
# в объект DataFrame
newdata = pd.read_csv('Data/Verizon_new.csv', 
                      encoding='utf-8', sep=';')

In [34]:
# применяем функцию предварительной обработки 
# к новым данным
preprocessing(newdata)

In [35]:
# применяем трансформеры с оптимальными значениями гиперпараметров, 
# обученные на всех исторических данных, к новым данным
newdata = ct_full.transform(newdata)

In [36]:
# проверяем размерность
newdata.shape

(100, 31)

In [37]:
# вычисляем при помощи логистической регрессии с оптимальными значениями 
# гиперпараметров, обученной на всех исторических данных,
# вероятности классов для новых данных
prob = logreg_full.predict_proba(newdata)
# выведем вероятности для первых 5 наблюдений
prob[:5]

array([[0.01697295, 0.98302705],
       [0.08082034, 0.91917966],
       [0.9626282 , 0.0373718 ],
       [0.96233072, 0.03766928],
       [0.09299023, 0.90700977]])